***GENERATED CODE FOR seereda PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run seeredaHooks.ipynb
try:
	#sourcePreExecutionHook()

	seersamplecopy = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1714378617752_SEERSample_copy1.csv', 'filename': 'SEERSample_copy1.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'FilePath': '/Khalid/dataFolder/SEERSample_copy1.csv', 'viewFileName': 'SEERSample_copy1.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run seeredaHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(seersamplecopy,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "CASENUM", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "42481035.57", "stddev": "20425661.31", "min": "3630565", "max": "93365819", "missing": "0"}, "updatedLabel": "CASENUM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MAR_STAT", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.33", "stddev": "0.47", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "MAR_STAT"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RACE", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.91", "stddev": "0.29", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "RACE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AGE_DX", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "36.71", "stddev": "9.03", "min": "17", "max": "71", "missing": "0"}, "updatedLabel": "AGE_DX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DATE_yr", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2000.84", "stddev": "1.57", "min": "1998", "max": "2003", "missing": "0"}, "updatedLabel": "DATE_yr"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "HISTO3V", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.99", "stddev": "0.11", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "HISTO3V"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EOD10_SZ", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "41.5", "stddev": "23.8", "min": "1", "max": "160", "missing": "0"}, "updatedLabel": "EOD10_SZ"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EOD10_EX", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "15.72", "stddev": "14.4", "min": "10", "max": "99", "missing": "0"}, "updatedLabel": "EOD10_EX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TUMOR_1V", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.66", "stddev": "0.78", "min": "0", "max": "4", "missing": "0"}, "updatedLabel": "TUMOR_1V"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TUMOR_2V", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.02", "stddev": "1.13", "min": "0", "max": "6", "missing": "0"}, "updatedLabel": "TUMOR_2V"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TUMOR_3V", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.04", "stddev": "1.37", "min": "0", "max": "6", "missing": "0"}, "updatedLabel": "TUMOR_3V"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SURGPRIM", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "42.06", "stddev": "9.35", "min": "30", "max": "80", "missing": "0"}, "updatedLabel": "SURGPRIM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RADIATN", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.21", "stddev": "1.06", "min": "1", "max": "7", "missing": "0"}, "updatedLabel": "RADIATN"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RAD_SURG", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.92", "stddev": "0.5", "min": "0", "max": "4", "missing": "0"}, "updatedLabel": "RAD_SURG"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "o_dth_cl", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.04", "stddev": "0.2", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "o_dth_cl"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SURV_TM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "7.32", "stddev": "1.86", "min": "0.25", "max": "10.91", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SURV_TM"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run seeredaHooks.ipynb
try:
	#sinkPreExecutionHook()

	datafolderoutput = NumtraConnector.fetch(spark, "{'samplefile': {'status': False}, 'samplecount': 210, 'originalcount': 2025, 'orignalKey': '/FileStore/platform/testdata/1714378617752_SEERSample_copy1.csv', 'pathOnly': '/Khalid/dataFolder', 'project_id': '662f576be3c6acbde8a874e5', 'parent_id': '662f576be3c6acbde8a874e5', 'original_schema': [], 'actual_schema': [], 'server': 'https://dnm.bfirst.ai:443', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'AutoFEoutput.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-5f82dd51-818f-4134-8830-f9a82636ac50-c000.csv', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
